# Exercise 2.03: Preparing the Quandl Data for Prediction

The goal of this exercise is to download an external dataset from the Quandl library and then prepare it so that it is ready for use in our linear regression models.

The following steps will help you to complete this exercise:

  > **Note**  
  > You need to install Qaundl library

1.- Download the data into a DataFrame using Quandl for the S&P 500 between 1950 and 2019. Its ticker is `“YALE/SPCOMP”`:

In [1]:
import quandl
import numpy as np
from sklearn import preprocessing
from sklearn import model_selection

2.- Use the `head()` method to visualize the columns inside the `data_frame.head()` DataFrame.

Output:

![Figure 2.18](img/fig2_18.jpg)

In [2]:
data_frame = quandl.get("YALE/SPCOMP", start_date="1950-01-01", end_date="2019-12-31")
data_frame.head()

,S&P Composite,Dividend,Earnings,CPI,Long Interest Rate,Real Price,Real Dividend,Real Earnings,Cyclically Adjusted PE Ratio
Year,,,,,,,,,
1950-01-31,16.88,1.15,2.33667,23.5,2.320000,187.229907,12.755592,25.917921,10.745733
1950-02-28,17.21,1.16,2.35333,23.5,2.340833,190.890208,12.866510,26.102711,10.911564
1950-03-31,17.35,1.17,2.37000,23.6,2.361667,191.627626,12.922439,26.176223,10.910947
1950-04-30,17.84,1.18,2.42667,23.6,2.382500,197.039587,13.032887,26.802133,11.178022
1950-05-31,18.44,1.19,2.48333,23.7,2.403333,202.807127,13.087879,27.312203,11.461543


A few features seem to highly correlate with each other. For instance, the `Real Dividend` column grows proportionally with `Real Price`. The ratio between them is not always similar, but they do correlate.

As regression is not about detecting the correlation between features, we would rather get rid of the features that we know are correlated and perform regression on the features that are non-correlated. In this case, we will keep the `Long Interest Rate`, `Real Price`, and `Real Dividend` columns.

3.- Keep only the relevant columns in the `Long Interest Rate`, `Real Price`, and `Real Dividend` DataFrames:

Output:

![Figure 2.19](img/fig2_19.jpg)

In [3]:
data_frame = data_frame[['Long Interest Rate', 'Real Price', 'Real Dividend']]
data_frame

,Long Interest Rate,Real Price,Real Dividend
Year,,,
1950-01-31,2.320000,187.229907,12.755592
1950-02-28,2.340833,190.890208,12.866510
1950-03-31,2.361667,191.627626,12.922439
1950-04-30,2.382500,197.039587,13.032887
1950-05-31,2.403333,202.807127,13.087879
...,...,...,...
2019-08-31,1.630000,2943.799674,57.747370
2019-09-30,1.700000,3027.438466,58.088856
2019-10-31,1.710000,3015.999351,58.300732


You can see that the DataFrame contains a few missing values `NaN`. As regression doesn't work with missing values, we need to either replace them or delete them. In the real world, we will usually choose to replace them. In this case, we will replace the missing values by the preceding values using a method called **forward filling**.

4.- Replace the missing values with a forward filling.

Output:

![Figure 2.20](img/fig2_20.jpg)

In [4]:
data_frame.fillna(method='ffill', inplace=True)
data_frame

,Long Interest Rate,Real Price,Real Dividend
Year,,,
1950-01-31,2.320000,187.229907,12.755592
1950-02-28,2.340833,190.890208,12.866510
1950-03-31,2.361667,191.627626,12.922439
1950-04-30,2.382500,197.039587,13.032887
1950-05-31,2.403333,202.807127,13.087879
...,...,...,...
2019-08-31,1.630000,2943.799674,57.747370
2019-09-30,1.700000,3027.438466,58.088856
2019-10-31,1.710000,3015.999351,58.300732


Now that we have cleaned the missing data, we need to create our label. We want to predict the `Real Price` column 3 months in advance using the current `Real Price`, `Long Interest` Rate, and `Real Dividend` columns. In order to create our label, we need to shift the `Real Price` values up by three units and call it `Real Price Label`.

6.- Create the `Real Price Label` label by shifting `Real Price` by 3 months.

Output:

![Figure 2.21](img/fig2_21.jpg)

In [5]:
data_frame['Real Price Label'] = data_frame['Real Price'].shift(-3)
data_frame

,Long Interest Rate,Real Price,Real Dividend,Real Price Label
Year,,,,
1950-01-31,2.320000,187.229907,12.755592,197.039587
1950-02-28,2.340833,190.890208,12.866510,202.807127
1950-03-31,2.361667,191.627626,12.922439,205.240598
1950-04-30,2.382500,197.039587,13.032887,187.976419
1950-05-31,2.403333,202.807127,13.087879,197.692277
...,...,...,...,...
2019-08-31,1.630000,2943.799674,57.747370,3146.548400
2019-09-30,1.700000,3027.438466,58.088856,3222.288571
2019-10-31,1.710000,3015.999351,58.300732,NaN


The side effect of shifting these values is that missing values will appear in the last three rows for `Real Price Label`, so we need to remove the last three rows of data. However, before that, we need to convert the features into a NumPy array and scale it. We can use the `drop` method of the DataFrame to remove the label column and the preprocessing function from `sklearn` to scale the features.

6.- Create a NumPy array for the features and scale it.

Output:

```
array([[-1.14834808, -1.13030753, -1.19145337],
       [-1.14109376, -1.12504171, -1.17960932],
       [-1.13383944, -1.12398084, -1.17363711],
       ...,
       [-1.36075461,  2.93924962,  3.67194058],
       [-1.32593387,  3.12706161,  3.7120735 ],
       [-1.3085235 ,  3.23602381,  3.75460521]])
```

In [6]:
features = np.array(data_frame.drop('Real Price Label', 1))
scaled_features = preprocessing.scale(features)
scaled_features

array([[-1.14834808, -1.13030753, -1.19145337],
       [-1.14109376, -1.12504171, -1.17960932],
       [-1.13383944, -1.12398084, -1.17363711],
       ...,
       [-1.36075461,  2.93924962,  3.67194058],
       [-1.32593387,  3.12706161,  3.7120735 ],
       [-1.3085235 ,  3.23602381,  3.75460521]])

The `1 in the second argument specifies that we are dropping columns. As the original DataFrame was not modified, the label can be directly extracted from it. Now that the features are scaled, we need to remove the last three values of the features as they are the features of the missing values in the label column. We will save them for later in the prediction part.

7.- Remove the last three values of the features array and save them into another array

Output:

```
array([[-1.14834808, -1.13030753, -1.19145337],
       [-1.14109376, -1.12504171, -1.17960932],
       [-1.13383944, -1.12398084, -1.17363711],
       ...,
       [-1.07522452,  2.86136628,  3.47166907],
       [-1.12049149,  2.78043332,  3.50007195],
       [-1.23888201,  2.83151407,  3.54006023]])
```

In [7]:
scaled_features_latest_3 = scaled_features[-3:]
scaled_features = scaled_features[:-3]
scaled_features

array([[-1.14834808, -1.13030753, -1.19145337],
       [-1.14109376, -1.12504171, -1.17960932],
       [-1.13383944, -1.12398084, -1.17363711],
       ...,
       [-1.38861121,  2.97929738,  3.57121717],
       [-1.38861121,  2.83538088,  3.61285159],
       [-1.36423669,  2.95570629,  3.64931605]])

The `scaled_features` variable doesn't contain the three data points anymore as they are now in `scaled_features_latest_3`. Now we can remove the last three rows with missing data from the DataFrame, then convert the label into a NumPy array using `sklearn`.

8.- Remove the rows with missing data

Output:

![Figure 2.22](img/fig2_22.jpg)

In [8]:
data_frame.dropna(inplace=True)
data_frame

,Long Interest Rate,Real Price,Real Dividend,Real Price Label
Year,,,,
1950-01-31,2.320000,187.229907,12.755592,197.039587
1950-02-28,2.340833,190.890208,12.866510,202.807127
1950-03-31,2.361667,191.627626,12.922439,205.240598
1950-04-30,2.382500,197.039587,13.032887,187.976419
1950-05-31,2.403333,202.807127,13.087879,197.692277
...,...,...,...,...
2019-05-31,2.400000,2905.605351,56.691201,2943.799674
2019-06-30,2.060000,2941.111837,57.065687,3027.438466
2019-07-31,1.630000,3043.836752,57.357468,3015.999351


9.- Now let's see if we have accurately created our label.

Output:

![Figure 2.23](img/fig2_23.jpg)

In [9]:
label = np.array(data_frame['Real Price Label'])
label

array([ 197.03958729,  202.807127  ,  205.24059811,  187.97641888,
        197.69227706,  203.82581434,  210.53941026,  209.26490617,
        205.9196225 ,  217.65948337,  223.13114786,  218.52818343,
        221.45805736,  220.70364701,  216.87932481,  220.70364701,
        230.36509257,  234.49210613,  232.4032458 ,  224.2249054 ,
        230.26407274,  237.93626311,  235.38485029,  235.97950675,
        234.39450701,  234.29577301,  239.80513   ,  244.84256067,
        245.81880693,  241.91382191,  236.83734139,  244.35443755,
        254.21452472,  256.54210132,  254.36261943,  254.68026024,
        242.13733092,  242.49956966,  232.93854897,  236.24540103,
        236.33615325,  225.48348857,  231.40615806,  237.40203996,
        240.59970009,  246.70432398,  252.13065632,  257.46008987,
        268.73035942,  278.3902289 ,  280.61890112,  291.95605976,
        297.76998727,  305.88381483,  312.98382071,  325.23862537,
        341.39331526,  347.54366667,  359.1609971 ,  356.32988

Our variable contains all the labels and is exactly the same as the `Real Price Label` column in the DataFrame.

Our next task is to separate the training and testing data from each other. As we saw in the Splitting Data into Training and Testing section, we will use $90\%$ of the data as the training data and the remaining $10\%$ as the test data.

10.- Split the `features` data into training and test sets using `sklearn`.

In [10]:
from sklearn import model_selection
(features_train, features_test, label_train, label_test) = model_selection.train_test_split(scaled_features, label, test_size=0.1, random_state=8)

By completing this exercise, we have learned all the required steps for data preparation before performing a regression.